In [1]:
import numpy as np

# mne library
import mne
from mne import  read_epochs
from mne.decoding import Vectorizer
mne.set_log_level('error') # Avoid long log

# Sklearn library
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression



In [3]:
# The path to the folder with the data
path = 'files/prepocessed_ica'

# Create a list with each tipe of experimental run
openeye_runs = [1]
closedeye_runs = [2]
fists_runs = [3, 7, 11]
imaginefists_runs = [4, 8, 12]
fistsfeet_runs = [5, 9, 13]
imaginefistsfeet_run = [6, 10, 14]

# List of participants
participants = [i for i in range(1,57)]


# Here we charge the data, the idea is to keep the data of each subject together. 
# So we will create a list that contain a list for each subject, which containts all the epochs of the three experiences.
data = []
labels = []


for participant in participants:
    participant_data = []
    participant_labels = []
    for run in fists_runs:
        epochs = read_epochs(path + f'/S{participant:03}R{run:02}filt01_30_ICA_masref.fif')
        participant_data.append(epochs.get_data())
        participant_labels.append(epochs.events[:,-1])
    data.append(np.concatenate(participant_data, axis = 0))
    labels.append(np.concatenate(participant_labels, axis = 0))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=0)

X_train = np.concatenate(X_train, axis = 0)
X_test = np.concatenate(X_test, axis = 0)
y_train = np.concatenate(y_train, axis = 0)
y_test = np.concatenate(y_test, axis = 0)

print('X_train dimensions: ', X_train.shape)
print('y_train dimensions: ', y_train.shape)
print('X_test dimensions: ', X_test.shape)
print('y_test dimensions: ', y_test.shape)

X_train, y_train = shuffle(X_train, y_train, random_state = 42)

X_train dimensions:  (3571, 64, 817)
y_train dimensions:  (3571,)
X_test dimensions:  (947, 64, 817)
y_test dimensions:  (947,)


In [5]:
lr_pip = make_pipeline(Vectorizer(), StandardScaler(), LogisticRegression(random_state=42))
parameters = {'logisticregression__penalty': ['l1', 'l2', 'elasticnet', None], 'logisticregression__solver' : ['newton-cg', 'sag', 'saga', 'lbfgs']}
gs_lr_pip = GridSearchCV(lr_pip, parameters, scoring='accuracy', verbose=3)

In [ ]:
gs_lr_pip.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END logisticregression__penalty=l1, logisticregression__solver=newton-cg;, score=nan total time=  12.9s
[CV 2/5] END logisticregression__penalty=l1, logisticregression__solver=newton-cg;, score=nan total time=   3.3s
[CV 3/5] END logisticregression__penalty=l1, logisticregression__solver=newton-cg;, score=nan total time=   2.8s
[CV 4/5] END logisticregression__penalty=l1, logisticregression__solver=newton-cg;, score=nan total time=   2.8s
[CV 5/5] END logisticregression__penalty=l1, logisticregression__solver=newton-cg;, score=nan total time=   2.8s
[CV 1/5] END logisticregression__penalty=l1, logisticregression__solver=sag;, score=nan total time=   2.8s
[CV 2/5] END logisticregression__penalty=l1, logisticregression__solver=sag;, score=nan total time=   2.8s
[CV 3/5] END logisticregression__penalty=l1, logisticregression__solver=sag;, score=nan total time=   2.8s
[CV 4/5] END logisticregression__penalty=l1, logistic

/home/elia/anaconda3/envs/mne/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 1/5] END logisticregression__penalty=l1, logisticregression__solver=saga;, score=0.635 total time=11.6min


/home/elia/anaconda3/envs/mne/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 2/5] END logisticregression__penalty=l1, logisticregression__solver=saga;, score=0.625 total time=11.4min


...

/home/elia/anaconda3/envs/mne/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 4/5] END logisticregression__penalty=None, logisticregression__solver=saga;, score=0.616 total time= 5.5min


/home/elia/anaconda3/envs/mne/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 5/5] END logisticregression__penalty=None, logisticregression__solver=saga;, score=0.613 total time= 5.2min
[CV 1/5] END logisticregression__penalty=None, logisticregression__solver=lbfgs;, score=0.597 total time=  45.6s
[CV 2/5] END logisticregression__penalty=None, logisticregression__solver=lbfgs;, score=0.560 total time=  46.4s
[CV 3/5] END logisticregression__penalty=None, logisticregression__solver=lbfgs;, score=0.580 total time=  45.9s
[CV 4/5] END logisticregression__penalty=None, logisticregression__solver=lbfgs;, score=0.606 total time=  47.6s
[CV 5/5] END logisticregression__penalty=None, logisticregression__solver=lbfgs;, score=0.598 total time=  42.2s


/home/elia/anaconda3/envs/mne/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
35 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/elia/anaconda3/envs/mne/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/elia/anaconda3/envs/mne/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/elia/anaconda3/envs/mne/lib/python3.12/site-packages/sklea

In [88]:
y_pred = gs_lr_pip.predict(X_test)

In [89]:
print(sum(y_test == y_pred))

296


In [90]:
print(sum(y_test == y_pred)/len(y_test))


0.6205450733752621
